In [1]:

import time
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import matplotlib.image as mpimg
from collections import OrderedDict
import pandas as pd

from skimage import io, transform
from math import *
import xml.etree.ElementTree as ET 
import pandas as pd
from skimage.transform import AffineTransform, warp
from skimage.transform import rotate as rotate_transform
from skimage.util import random_noise
from skimage.filters import gaussian
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle


In [2]:
# Load models

class vgg16_bn_(nn.Module):
    def __init__(self,num_classes=1):
        super().__init__()
        self.model_name='vgg16_bn'
        self.model=models.vgg16_bn(pretrained=True)
        #self.model.conv1=nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.classifier=nn.Linear(self.model.classifier[0].in_features, num_classes)
        
    def forward(self, x):
        x = torch.sigmoid(self.model(x))
        return x

network = vgg16_bn_()
network.load_state_dict(torch.load('D:/Tsetse fly Project/Data/classifiers/models/model_vgg16_bn_classifer_finetune.pth'))
network.cuda()
network.eval()

vgg16_bn_(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [ ]:
'''
# V20
pages_dir = 'F:/tsetse wings/V20rename/V20/'
pages = os.listdir('F:/tsetse wings/V20rename/V20')

results = pd.DataFrame([], columns= ['goodwings', 'badwings'])
for page in pages:
    files_dir = 'F:/tsetse wings/V20rename/V20/page1/Picture/'
    files = os.listdir('F:/tsetse wings/V20rename/V20/page1/Picture')
    for filename in files:
        if filename[:3] == 'V20':
            with torch.no_grad():
                image = Image.open(files_dir + filename)
                image = TF.resize(image, (244,244))
                image = TF.to_tensor(image).cuda()
                image = TF.normalize(image, [0.5], [0.5])
                image = torch.reshape(image, (1, 3, 244, 244))
                prediction = round(network(image).item())
                if prediction == 0:
'''
 

In [ ]:

#@ Author Dylan Geldenhuys

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# You may use the morphometric_data.csv, and remove columns "left_good" and "right_good" to reproduce these results.
csv_dir = "D:/Tsetse fly Project/Data/Vols2021DataforMorphometricsOct2020.csv"
V20 = "F:/tsetse wings/V20/"
V21 = "F:/tsetse wings/V21/"
csv = pd.read_csv(csv_dir, sep=';')

pred_l = []
pred_r = []
counter = 0
start_time = time.time()
for row in range(len(csv)):
    counter += 1
    if counter == 7000:
        print('half way')
    #   V20
    if int(str(csv.iloc[row,0])[:2]) == 20 and (('V20P{}L{}L.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ))) and ('V20P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') )))):
        #filename_ = 'V20P{}L{}L.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:])
        #if filename_ in os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') )):

        im_L = Image.open(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V20P{}L{}L.bmp'.format                   (str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))

        with torch.no_grad():
            im_L = TF.resize(im_L, (244,244))
            im_L = TF.to_tensor(im_L).cuda()
            im_L = TF.normalize(im_L, [0.5], [0.5])
            im_L = torch.reshape(im_L, (1, 3, 244, 244))
            predictionL = round(network(im_L).item())
            pred_l.append(predictionL)
            

        im_R = Image.open(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V20P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))
        im_R = im_R.transpose(method=Image.FLIP_LEFT_RIGHT) 

        with torch.no_grad():
            im_R = TF.resize(im_R, (244,244))
            im_R = TF.to_tensor(im_R).cuda()
            im_R = TF.normalize(im_R, [0.5], [0.5])
            im_R = torch.reshape(im_R, (1, 3, 244, 244))
            predictionR = round(network(im_R).item())
            pred_r.append(predictionR)
    else:
        pred_l.append('miss')
        pred_r.append('miss')

    if int(str(csv.iloc[row,0])[:2]) == 21 and ('V21P{}L{}L.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ))) and ('V21P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ))):

        im_L = Image.open(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V21P{}L{}L.bmp'.format(str                   (csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))

        with torch.no_grad():
            im_L = TF.resize(im_L, (244,244))
            im_L = TF.to_tensor(im_L).cuda()
            im_L = TF.normalize(im_L, [0.5], [0.5])
            im_L = torch.reshape(im_L, (1, 3, 244, 244))
            predictionL = round(network(im_L).item())
            pred_l.append(predictionL)
            

        im_R = Image.open(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V21P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))
        im_R = im_R.transpose(method=Image.FLIP_LEFT_RIGHT) 

        with torch.no_grad():
            im_R = TF.resize(im_R, (244,244))
            im_R = TF.to_tensor(im_R).cuda()
            im_R = TF.normalize(im_R, [0.5], [0.5])
            im_R = torch.reshape(im_R, (1, 3, 244, 244))
            predictionR = round(network(im_R).item())
            pred_r.append(predictionR)
    else:
        #    print(filename_)
        #    print(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + filename_)
        #    print(os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') )))
        pred_l.append('miss')
        pred_r.append('miss')

csv['left_good'] = pred_l
csv['right_good'] = pred_r
csv.to_csv('results1.csv') 

print("Total Elapsed Time : {} s".format(time.time()-start_time))


In [5]:
#@ Author Dylan Geldenhuys

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# You may use the morphometric_data.csv, and remove columns "left_good" and "right_good" to reproduce these results.
csv_dir = "D:/Tsetse fly Project/Data/Vols2021DataforMorphometricsOct2020.csv"
V20 = "G:/tsetse wings/V20rename/V20/"
V21 = "G:/tsetse wings/V21rename/"
csv = pd.read_csv(csv_dir, sep=';')

pred_l = []
pred_r = []
counter = 0
start_time = time.time()
for row in range(len(csv)):
    counter += 1
    if counter == 7000:
        print('half way')
    #   V20
    if int(str(csv.iloc[row,0])[:2]) == 20: 
        if (('V20P{}L{}L.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ))) and ('V20P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') )))):
        #filename_ = 'V20P{}L{}L.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:])
        #if filename_ in os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') )):

            im_L = Image.open(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V20P{}L{}L.bmp'.format                   (str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))

            with torch.no_grad():
                im_L = TF.resize(im_L, (244,244))
                im_L = TF.to_tensor(im_L).cuda()
                im_L = TF.normalize(im_L, [0.5], [0.5])
                im_L = torch.reshape(im_L, (1, 3, 244, 244))
                predictionL = round(network(im_L).item())
                pred_l.append(predictionL)
                #print(predictionL)
                

            im_R = Image.open(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V20P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))
            im_R = im_R.transpose(method=Image.FLIP_LEFT_RIGHT) 


            with torch.no_grad():
                im_R = TF.resize(im_R, (244,244))
                im_R = TF.to_tensor(im_R).cuda()
                im_R = TF.normalize(im_R, [0.5], [0.5])
                im_R = torch.reshape(im_R, (1, 3, 244, 244))

                predictionR = round(network(im_R).item())
                pred_r.append(predictionR)
               
        else:
            pred_l.append('miss')
            pred_r.append('miss')

    elif int(str(csv.iloc[row,0])[:2]) == 21:
        if ('V21P{}L{}L.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ))) and ('V21P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]) in os.listdir(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ))):

            im_L = Image.open(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V21P{}L{}L.bmp'.format(str                   (csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))

            with torch.no_grad():
                im_L = TF.resize(im_L, (244,244))
                im_L = TF.to_tensor(im_L).cuda()
                im_L = TF.normalize(im_L, [0.5], [0.5])
                im_L = torch.reshape(im_L, (1, 3, 244, 244))
                predictionL = round(network(im_L).item())
                pred_l.append(predictionL)
                

            im_R = Image.open(V21 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + 'V21P{}L{}R.bmp'.format(str(csv.iloc[row,0])[-5:-2],str(csv.iloc[row,0])[-2:]))
            im_R = im_R.transpose(method=Image.FLIP_LEFT_RIGHT) 


            with torch.no_grad():
                im_R = TF.resize(im_R, (244,244))
                im_R = TF.to_tensor(im_R).cuda()
                im_R = TF.normalize(im_R, [0.5], [0.5])
                im_R = torch.reshape(im_R, (1, 3, 244, 244))
                predictionR = round(network(im_R).item())
                pred_r.append(predictionR)
        else:
            #    print(filename_)
            #    print(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') ) + filename_)
            #    print(os.listdir(V20 + 'page{}/Picture/'.format( str(csv.iloc[row,0])[-5:-2].lstrip('0') )))
            pred_l.append('miss')
            pred_r.append('miss')

csv['left_good'] = pred_l
csv['right_good'] = pred_r
csv.to_csv('C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/Pipeline/Results/Results_2021_03_28/results_final.csv') 
print("Total Elapsed Time : {} s".format(time.time()-start_time))



half way
Total Elapsed Time : 1706.011735200882 s


In [6]:
csv

,vpn,cd,cm,cy,md,g,s,c,wlm,f,lmkl,lmkr,hc,left_good,right_good
0,20000101,4,November,94,Odour-baited trap,Gp,2,1.0,732,3,0,0,0,0,0
1,20000102,4,November,94,Odour-baited trap,Gp,2,5.0,732,4,0,0,0,0,0
2,20000103,4,November,94,Odour-baited trap,Gp,2,5.0,723,5,0,0,0,0,0
3,20000104,4,November,94,Odour-baited trap,Gp,2,4.0,732,5,0,0,0,0,0
4,20000105,4,November,94,Odour-baited trap,Gp,2,7.0,713,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14663,21077806,21,July,96,Odour-baited trap,Gp,2,0.0,771,1,0,0,0,0,0
14664,21077807,21,July,96,Odour-baited trap,Gp,2,2.0,752,3,0,0,0,0,0
14665,21077808,21,July,96,Odour-baited trap,Gp,2,2.0,752,2,0,0,0,0,0
14666,21077809,21,July,96,Odour-baited trap,Gp,2,1.0,761,2,0,0,0,0,0
